In [ ]:
import requests
import pandas as pd
import time

API_KEY = 'a1126786c2894039926b5761c977cf82'
RECIPE_COUNT = 10000  # Increase for more recipes

def get_recipes(number):
    url = f"https://api.spoonacular.com/recipes/random?number={number}&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code != 200:
        print("Error:", response.json())
        return []
    return response.json().get('recipes', [])

def parse_recipes(recipes):
    data = []
    for r in recipes:
        try:
            title = r.get('title')
            ingredients = ', '.join([i['original'] for i in r['extendedIngredients']])
            nutrition = {n['title'].lower(): n['amount'] for n in r.get('nutrition', {}).get('nutrients', [])}

            data.append({
                'Title': title,
                'Ingredients': ingredients,
                'Calories': nutrition.get('calories'),
                'Protein (g)': nutrition.get('protein'),
                'Carbs (g)': nutrition.get('carbohydrates'),
                'Fiber (g)': nutrition.get('fiber')
            })
        except Exception as e:
            print(f"Skipping due to error: {e}")
            continue
    return data

# Collect recipes in batches of 10
final_data = []
for _ in range(RECIPE_COUNT // 10):
    batch = get_recipes(10)
    final_data.extend(parse_recipes(batch))
    time.sleep(2)  # Respect API rate limits

# Save to CSV
df = pd.DataFrame(final_data)
df.to_csv("spoonacular_recipes.csv", index=False)
print("Saved to spoonacular_recipes.csv")
